<a href="https://colab.research.google.com/github/namanphy/END2/blob/main/S6/EncDec_LSTM_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialising my wandb

In [ ]:
! pip install --upgrade wandb --quiet

     |████████████████████████████████| 1.8MB 2.8MB/s 
     |████████████████████████████████| 174kB 18.7MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 133kB 18.9MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 


In [ ]:
! pip install OmegaConf --quiet

In [ ]:
from omegaconf import OmegaConf

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: namanphy (use `wandb login --relogin` to force relogin)


True

In [ ]:
wand

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [ ]:
import pandas as pd
df = pd.read_csv('/content/tweets.csv')
df.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [ ]:
df.shape

(1364, 2)

In [ ]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [ ]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [ ]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [ ]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [ ]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [ ]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [ ]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'Admit',
  'women',
  'to',
  'Augusta',
  'golf',
  'club',
  ':',
  'US',
  'President',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://t.co/PVKrepqI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [ ]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [ ]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(None, {0: 0, 1: 1, 2: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

In [ ]:
item = next(iter(train_iterator))
print(item.tweets[0].shape)
print(item.tweets[1].shape)

torch.Size([32, 8])
torch.Size([32])


Save the vocabulary for later use

In [ ]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

# Config

In [ ]:
hparams = OmegaConf.create({
    'epochs':10,
    'batch_size':32,
    'learning_rate':0.0001,
    'input_dim':len(Tweet.vocab),
    'embedding_dim':300,
    'enc_hidden_dim':150,
    'dec_hidden_dim':150,
    'output_dim':len(Label.vocab),
    'dropout': 0.3,
})

In [ ]:
wandb.init(project='Encoder-Decoder-lstm-classification')
config = wandb.config
config = hparams
config

{'epochs': 10, 'batch_size': 32, 'learning_rate': 0.0001, 'input_dim': 4651, 'embedding_dim': 300, 'enc_hidden_dim': 150, 'dec_hidden_dim': 150, 'output_dim': 3, 'dropout': 0.3}

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).printppp

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class lstm_encoder(nn.Module):

    def __init__(self, vocab_size, embedding_size, hidden_size, n_layers, dropout):
        super(lstm_encoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=n_layers, dropout=dropout, batch_first=True)

    def forward(self, input, input_lengths):
        print('Shapes Insider Encoders :')
        print('\t Input Length : ', len(input), '\n\t\t Data : ', input.data.shape)
        print('\t Input Lengths : ', input_lengths.shape)

        embedded_input = self.embedding(input)
        packed_embedded_input = nn.utils.rnn.pack_padded_sequence(embedded_input, input_lengths.cpu(), batch_first=True)

        print('\t Embeddings : ', embedded_input.shape, '\n\t\t Packed Embedded inout : ', packed_embedded_input.data.shape)
                
        output, (hidden, cell) = self.lstm(packed_embedded_input)

        return output, (hidden, cell)


In [ ]:
class lstm_decoder(nn.Module):

    def __init__(self, input_size, hidden_size, dropout=0):
        super(lstm_decoder, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, dropout=dropout)

    def forward(self, input, hidden_encoder, cell_encoder):
        output, (hidden, cell) = self.lstm(input, (hidden_encoder, cell_encoder))

        return output, (hidden, cell)

In [ ]:
class classifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, batch_size, n_layers, dropout):        
        super(classifier, self).__init__()
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        self.encoder = lstm_encoder(vocab_size, embedding_dim, hidden_dim, n_layers, dropout)
        self.decoder = lstm_decoder(embedding_dim, hidden_dim, dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        encoder_output, (encoder_hidden, encoder_cell) = self.encoder(text, text_lengths)

        print('Shapes of Encoders :')
        print('\t Output Length : ', len(encoder_output), '\n\t\t Data : ', encoder_output.data.shape, 
              '\n\t\t Batch size : ', encoder_output.batch_sizes)
        print('\t Hidden : ', encoder_hidden.shape)
        print('\t Cell Length : ', len(encoder_cell), '\n\t\t Data Shape : ', encoder_cell.data.shape, 
              '\n\t\t Data : ', encoder_cell)

        print("-------------------------------------------")
        
        decoder_first_input = self.get_decoder_input(encoder_hidden.shape) # pass decoder hidden shape
        decoder_output, (decoder_hidden, decoder_cell) = self.decoder(decoder_first_input, encoder_hidden, encoder_cell)
        
        print('Shapes of Decoders :')
        print('\t Output Length : ', len(decoder_output), '\n\t\t Data : ', decoder_output.data.shape)
        print('\t Hidden : ', decoder_hidden.shape)
        print('\t Cell Length : ', len(decoder_cell), '\n\t\t Data Shape : ', decoder_cell.data.shape, 
              '\n\t\t Data : ', decoder_cell)
        print("\n\n")
        final_outputs = self.fc(decoder_hidden)
        return final_outputs

    def get_decoder_input(self, hidden_shape, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        return torch.zeros(1, hidden_shape[1], self.embedding_dim).to(device)
    

In [ ]:
config

{'epochs': 10, 'batch_size': 32, 'learning_rate': 0.0001, 'input_dim': 4651, 'embedding_dim': 300, 'enc_hidden_dim': 150, 'dec_hidden_dim': 150, 'output_dim': 3, 'dropout': 0.3}

In [ ]:
# # Define hyperparameters
# size_of_vocab = len(Tweet.vocab)
# embedding_dim = 300
# num_hidden_nodes = 100
# num_output_nodes = 3
# num_layers = 1
# dropout = 0.2

# # Instantiate the model
# model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, 32, num_layers, dropout = dropout)

model = classifier(config.input_dim, config.embedding_dim, config.enc_hidden_dim, config.output_dim, config.batch_size,
                   1, config.dropout)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (encoder): lstm_encoder(
    (embedding): Embedding(4651, 300)
    (lstm): LSTM(300, 150, batch_first=True, dropout=0.3)
  )
  (decoder): lstm_decoder(
    (lstm): LSTM(300, 150, dropout=0.3)
  )
  (fc): Linear(in_features=150, out_features=3, bias=True)
)
The model has 1,938,153 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [ ]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [ ]:
N_EPOCHS = config.epochs
best_valid_loss = float('inf')

wandb.watch(model, criterion, log='all', log_freq=10)

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    wandb.log({
        'train loss': train_loss,
        'train acc': train_acc*100,
        'val loss': valid_loss,
        'val acc': valid_acc*100
    })
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Streaming output truncated to the last 5000 lines.
	 Input Lengths :  torch.Size([32])
	 Embeddings :  torch.Size([32, 28, 300]) 
		 Packed Embedded inout :  torch.Size([884, 300])
Shapes of Encoders :
	 Output Length :  4 
		 Data :  torch.Size([884, 150]) 
		 Batch size :  tensor([32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 20])
	 Hidden :  torch.Size([1, 32, 150])
	 Cell Length :  1 
		 Data Shape :  torch.Size([1, 32, 150]) 
		 Data :  tensor([[[-0.6561,  0.6684, -1.2487,  ...,  0.4775, -0.5191, -1.7716],
         [-0.2995,  1.6167, -0.6769,  ..., -0.5065, -1.1548, -0.3950],
         [-0.4268,  0.4422, -0.3564,  ..., -0.2738, -1.2414,  0.1167],
         ...,
         [-0.6158,  1.0526, -0.5168,  ..., -0.7725, -1.0073, -1.0402],
         [-0.9183,  0.6789, -0.9880,  ..., -0.0736, -0.6529, -0.5154],
         [-0.8821,  1.9856,  0.0292,  ..., -1.4550, -1.9144, -0.6516]]],
       device='cuda:0', grad_fn=<CudnnRnnB

## Model Testing

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
import random
import googletrans
import googletrans.Translator

translator = Translator()
sentence = ['']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)